In [1]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, Lambda
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_curve, auc
import scipy
from PIL import Image
import io
import pathlib
import os
import cv2

C:\Users\Asus\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [ ]:
# for unclassified img use classifier to generate X and Y array then split

In [2]:
#train array
x1 = []
y1 = []
path = 'Include/train/'
for b in ['negative', 'positive']:
    name_lst = os.listdir(path + b)
    for img_name in name_lst:
        img = cv2.imread(path + b + '/' + img_name)
        img1 = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        x1.append(img1)
        y1.append(b)
        
X_train = np.array(x1)
Y_train = np.array(y1)

In [3]:
#test array (already classified)
x2 = []
y2 = []
path = 'Include/test/'
for b in ['negative', 'positive']:
    name_lst = os.listdir(path + b)
    for img_name in name_lst:
        img = cv2.imread(path + b + '/' + img_name)
        img1 = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        x2.append(img1)
        y2.append(b)
        
X_test = np.array(x2)
Y_test = np.array(y2)

In [ ]:
# split train and test in array
# X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 10)

In [4]:
negative_train = X_train[np.where(Y_train == 'negative')]
positive_train = X_train[np.where(Y_train == 'positive')]
negative_test = X_test[np.where(Y_test == 'negative')]
positive_test = X_test[np.where(Y_test == 'positive')]

In [5]:
datagen = ImageDataGenerator(
    rotation_range=20,
    fill_mode='constant',
    height_shift_range=0.1,
    width_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    shear_range=0.2)

In [6]:
# train generator
negative_train_generator = datagen.flow(negative_train,
                                Y_train[np.where(Y_train == 'negative')],
                                save_to_dir = 'Include/train/negative',
                                batch_size = 1)
positive_train_generator = datagen.flow(positive_train,
                                Y_train[np.where(Y_train == 'positive')],
                                save_to_dir = 'Include/train/positive',
                                batch_size = 1)

In [11]:
print(len(X_test))
print(len(Y_test))

188
188


In [12]:
# test generator
negative_test_generator = datagen.flow(negative_test,
                                Y_train[np.where(Y_test == 'negative')],
                                save_to_dir = 'Include/test/negative',
                                batch_size = 1)
positive_test_generator = datagen.flow(positive_test,
                                Y_train[np.where(Y_test == 'positive')],
                                save_to_dir = 'Include/test/positive',
                                batch_size = 1)

In [8]:
print(len(X_train))
print(len(Y_train))
print(len(X_test))

750
750
188


In [9]:
# run train generator
for i in range(len(negative_train)):
    next(negative_train_generator)
for i in range(len(positive_train)):
    next(positive_train_generator)

In [13]:
# run test generator
for i in range(len(negative_test)):
    next(negative_test_generator)
for i in range(len(positive_test)):
    next(positive_test_generator)

In [15]:
# save X_train, Y_train
x = []
y = []

path = 'Include/train/'
for b in ['negative', 'positive']:
    name_lst = os.listdir(path + b)
    for img_name in name_lst:
        img = cv2.imread(path + b + '/' + img_name)
        img1 = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        x.append(img1)
        y.append(b)
        
X = np.array(x)
Y = np.array(y)

np.save('X_train.npy', X)
np.save('Y_train.npy', Y)

In [20]:
# save X_test, Y_test
v = []
w = []

path = 'Include/test/'
for b in ['negative', 'positive']:
    name_lst = os.listdir(path + b)
    for img_name in name_lst:
        img = cv2.imread(path + b + '/' + img_name)
        img1 = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        v.append(img1)
        w.append(b)
        
V = np.array(v)
W = np.array(w)

np.save('X_test.npy', V)
np.save('Y_test.npy', W)

In [19]:
# randomly choose 200 files from negative train dataset (equal to positive dataset)
import random
img_random_lst = random.sample(os.listdir('Include/train/negative'), 200)
r = []
rr = []

for img_name in img_random_lst:
    img = cv2.imread('Include/train/negative/' + img_name)
    img1 = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    r.append(img1)
    rr.append('negative')

positive_lst = os.listdir('Include/train/positive')
for img_name in positive_lst:
    img = cv2.imread('Include/train/positive/' + img_name)
    img1 = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    r.append(img1)
    rr.append('positive')

R = np.array(r)
RR = np.array(rr)

np.save('X_train_2.npy', R)
np.save('Y_train_2.npy', RR)

In [ ]:
print(len(X))
print(len(Y))
print(len(negative_train))
print(len(positive_train))
